<a href="https://colab.research.google.com/github/KravitzLab/KreedLabWiki/blob/main/Kravitz_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PCA plotting for PsyGene Mice Behavior

In [1]:
#!pip install igraph
#!pip install leidenalg
#!pip install scanpy

In [96]:
#@title # Import Libraries

####### Import Needed Libraries #######
# Libraries for Machine Learnings/Modeling
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from scipy.optimize import linear_sum_assignment
# Libraries for importing
import zipfile
from google.colab import files # creates the ability for display uploads
import tempfile
import io
import os
# Data Frame libraries
import pandas as pd
import numpy as np
# for merging
from functools import reduce
# UMAP
import umap
# Plotting
import seaborn as sns
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
# Clustering
from sklearn.neighbors import kneighbors_graph
import igraph as ig
import leidenalg
import scanpy as sc


from decorator import DEF

import ipywidgets as widgets

In [145]:
#@title # Define functions used through out the script

####### DEFINE ALL FUNCTIONS #######

### Session type Extraction ###
# This code will extract the "session_type" column from each file
def extract_session_type(df, file_name, fallback="Unknown"):
    # Follow Naming conventions so far
    session_types = ["weight", "bandit100", "fr1", "beam", "bandit80",
                     "_pr_", # we have to define pr surrounded by underscores to prevent random pr's
                     "individual_behavoir", "social_interaction", "nesting"]
    """Read 'Session_Type ' or variants; return first non-empty value."""
    file_name = file_name.lower()
    print(f"Extracting session type for file: {file_name}")
    try:
        #df = pd.read_csv(csv_path, sep=None, engine='python', dtype=str)
        df = df
        df.columns = [c.strip() for c in df.columns]
        lower = {c.casefold(): c for c in df.columns}
        # Loop through common session like column names
        for cand in ["session_type", "session type", "sessiontype", "session"]:
            if cand in lower:
                col = lower[cand]
                vals = df[col].dropna().astype(str).str.strip()
                vals = vals[vals.ne("")]
                if not vals.empty:
                    return vals.iloc[0].lower()
            else:
              # use the file name to determine what session type it is
              matches = [c for c in session_types if c.lower() in file_name.lower()]
              return(matches[0])

    except Exception as e:
        print(f"Error reading {csv_path}: {e}")
    return fallback

### Retrieve dataframe profiles ###
# expects a dataframe as an argument
def df_prof(df):
  # Get Dimensions
  print(f"Number of rows: {df.shape[0]}")
  print(f"Number of columns: {df.shape[1]}")

  # Data types
  df_prof = df.dtypes.rename("DataType")

  # Null percentages
  df_null = (df.isna().sum() / len(df) * 100).rename("Percent_Null")

  # unique counts
  df_uniq = df.nunique().rename("Number_Unique")

  # Merge into one DataFrame
  merged_df = pd.concat([df_prof, df_null, df_uniq], axis=1).reset_index()
  merged_df.columns = ["Column", "DataType", "Percent_Null", "Number_Unique"]
  print(merged_df)



### Define easy function for finding mode ###
def mode_or_nan(x):
    # handle ties or empty groups safely
    return x.mode().iloc[0] if not x.mode().empty else None


### Aggregate Session Dataframes ###
# expects dictionary containing dataframes
# Requires "Mouse_ID" Column
# Will iterate through dataframes within the session dictionary passed
#     and aggregate by Mouse_ID and ensure only 1 mouse is represented.
# define this as a function for better readbility and to easily adjust
#   drop lists
def aggregate_session_types(session_dfs):
    # Iterate across the session dictionary
    for s, d in session_dfs.items():
        print(f"Aggregating Session Type: {s}")

        # Check to see if primary key is same length as number of rows in dataframe
        if len(d) == d["Mouse_ID"].nunique():
            print("Number of Mice equal to Length of dataframe: No Aggregation Needed")
            continue

        # Define numeric and categorical columns
        numeric_cols = d.select_dtypes(include=['int64', 'float64']).columns
        object_cols = d.select_dtypes(include=['object', 'category', 'datetime64']).columns

        # perform aggregation
        agg_dict = {
            **{col: 'mean' for col in numeric_cols},
            **{col: mode_or_nan for col in object_cols}
        }

        session_dfs[s] = (d.groupby('Mouse_ID', dropna=False, as_index=False)
                          .agg(agg_dict))
    return session_dfs



### Clean Session Dataframes ###
# Very fragile and expects the user to do manual investigation
# could add detection for columns with null values
# define this as a function for better readbility and to easily adjust
#   drop lists
def clean_session_types(session_dfs, all_drops, percent_drop = 5):
  # Interate through large table and remove
  for s, d in session_dfs.items():
    print(f"Cleaning Session Type: {s}")

    # remove "$" from column names
    session_dfs[s].columns = session_dfs[s].columns.str.replace('$', '')

    ### Conduct general data cleaning ###
    session_dfs[s] = session_dfs[s].drop(columns = all_drops, errors='ignore')


    ### set to lowercase ###
    session_dfs[s].columns = session_dfs[s].columns.str.lower()

    ### Drop based on nullness ###
    # Calculate the percentage of null values for each column
    null_percentages = session_dfs[s].isnull().sum() * 100 / len(session_dfs[s])

    # ensure that key columns are not dropped
    key_cols = ["mouse_id", "sex", "genotype", "gene"]

    # Identify columns where the null percentage is greater than 5%
    columns_to_drop = null_percentages[null_percentages > percent_drop].index

    # Remove key columns from the drop list
    columns_to_drop = [col for col in columns_to_drop if col not in key_cols]

    # Drop the identified columns from the DataFrame
    session_dfs[s] = session_dfs[s].drop(columns=columns_to_drop)

    ### Add suffix if doesnt exist ###
    # Columns to not add suffix
    exceptions = ["mouse_id", "sex", "genotype", "gene", "session_type"]

    session = session_dfs[s]["session_type"].iloc[0]
    session_dfs[s].rename(
    columns=lambda c: f"{c}_{session}"
                      if (c not in exceptions and c not in exceptions and not c.endswith(f"_{session}"))
                      else c,
    inplace=True
    )
    """
    ### Clean bandit80 specific data ###
    if s.lower() in ["bandit80", "bandit100"]:
      session_dfs[s] = session_dfs[s].drop(columns = bandit_drops, errors='ignore')
      # Ensure date times are correct data type
      session_dfs[s]['FED_StartDate'] = pd.to_datetime(session_dfs[s]['FED_StartDate'])

    ### Clean weight specific data ###
    if s.lower() in ["weight"]:

      session_dfs[s] = session_dfs[s].drop(columns = weight_drops, errors='ignore')
      # Ensure date times are correct data type
      session_dfs[s]['DOB'] = pd.to_datetime(session_dfs[s]['DOB'])

    # remove session type after all the cleaning
    session_dfs[s] = session_dfs[s].drop(columns = "session_type", errors='ignore')
  """
  ### Return the session_dfs ###
  return(session_dfs)





In [ ]:
#@title # Import Wanted Data Files
#@markdown Code based on Chantelle Murelle and Sebastian Alves\
#@markdown The following code will iteritvly read all csv's within the
#@markdown zipped file and seperate them into a dictionary based on the various
#@markdown recordings. this will be used for processing the data in a session type manner.\
#@markdown This could include L3 for all various recordings:
#@markdown 1.   Weight
#@markdown 2.   Bandit100_0
#@markdown 3.   FR1
#@markdown 4.   BEAM
#@markdown 5.   Bandit80_20
#@markdown 6.   PR
#@markdown 7.   Individual Behavoir
#@markdown 8.   Social Interaction
#@markdown 9.   Nesting




####### Import Wanted Data Files ########
# Code based on Chantelle Murelle and Sebastian Alves

### reset cache ###
# Reset caches to avoid duplicates if you re-run this cell
session_dict, loaded_files, session_types = [], [], []

### Define the upload UI element ###
uploaded = files.upload()

### Create Data dictonary and containers ###
session_dict = {} # for storing by session type
loaded_files = [] # for files

### Loop through the uploaded files ###
# This expects a zippped file exclusivly
for name, data in uploaded.items():
  with zipfile.ZipFile(io.BytesIO(data)) as zf: # use zipfile to "unzip"
    for zi in zf.infolist(): # iterate through the zipped files


      # skip non handled file types
      # Skip non-csv/xlsx files
      if not (zi.filename.endswith(".csv") or zi.filename.endswith(".xlsx")):
        print(f"Skipping: {zi.filename}")
        continue
      print(f"\nProcessing: {zi.filename}")

      # define filetype
      file_type = "csv" if zi.filename.endswith(".csv") else "xlsx"

      # read the file in
      file_data = zf.read(zi)

      # write temporary file into memory
      suffix = ".csv" if file_type == "csv" else ".xlsx"
      with tempfile.NamedTemporaryFile(mode="w+b", suffix=suffix, delete=False) as tmp:
        tmp.write(file_data)
        tmp_path = tmp.name

      # Define try catch statment and read into memory
      try:
        if file_type == "csv":
          df = pd.read_csv(tmp_path)
        else:
          df = pd.read_excel(tmp_path)
        #df["source_file"] = zi.filename
        session_type = extract_session_type(df, zi.filename)
        # rewrite session
        df["session_type"] = session_type
        print(session_type)

        # Add to the data dictionary based on session type
        if session_type not in session_dict:
          session_dict[session_type] = []
        session_dict[session_type].append(df)
      except Exception as e: # throw error on failure to load
        print(f"Error loading {zi.filename}: {e}")
      finally: # remove the temporary path
        os.remove(tmp_path)



### Combine dataframes for each session ###
# Convert each list of dfs into one merged dataframe per session type
session_dfs = {k: pd.concat(v, ignore_index=True) for k, v in session_dict.items()}

session_dfs.pop('weight', None)
#session_dfs.pop('bandit100', None)
session_dfs.pop('beam', None)
session_dfs.pop('si', None)

#print(session_dfs['bandit100'])

### Return overarcing view of all sessiont types ###
for s, d in session_dfs.items():
    print(f"\nSession '{s}' → {d.shape[0]} rows, {d.shape[1]} cols")
    df_prof(session_dfs[s])

In [ ]:
# @title # Begin to clean data.
# @markdown Define what columns for what session types will be dropped.
# @markdown This should be based on nullness and should seek to keep as
# @markdown large an N as possible. Additionally, columns that are believe
# @markdown to add little to no context could be dropped.

####### BEGIN DATA CLEANING AND SCRUBBING #######


# drop known problem columns
all_drops = ["filename", "filename_md", "Session_type", "match_status",
               "Primary", "Autoexcluder", "Gene_ID"]



dfs_agg = aggregate_session_types(session_dfs)


dfs_clean = clean_session_types(session_dfs=dfs_agg,
                                all_drops=all_drops)


### Return overarcing view of all sessiont types ###
for s, d in dfs_clean.items():
    print(f"\nSession '{s}' → {d.shape[0]} rows, {d.shape[1]} cols")
    df_prof(dfs_clean[s])




In [ ]:
# @title # Merge and Prepare Columns
# @markdown Continues cleaning and data formatting including:
# @markdown 1. Remove any rows that have any null values present.
# @markdown 2. Number of Days columns (if date times are present).




####### Merge the list of dataframes #######

# Define the features that will be merged on
merge_keys = ["mouse_id", "gene", "sex", "genotype"]

# Define columns for metadata
meta_rows = []

# ensure all key columns are normalized
for s, d in dfs_clean.items():
    dfs_clean[s][merge_keys] = dfs_clean[s][merge_keys].astype(str).apply(lambda col: col.str.upper())

    # Build meta data file
    # extract the session type (assume same for all rows)
    session = dfs_clean[s]["session_type"].iloc[0]
    # take all columns *except* session_type
    cols = dfs_clean[s].columns.drop("session_type")
    # add metadata rows
    for c in cols:
        meta_rows.append({"column_name": c, "session_type": session})

# Create the meta datafrane linking all session types to features
df_meta = pd.DataFrame(meta_rows)
df_meta = df_meta[~df_meta["column_name"].isin(merge_keys)]

df_meta = df_meta[['session_type', 'column_name']]
print(f"Meta data frame: \n{df_meta}")




# Merge files
merged = reduce(
    lambda left, right: pd.merge(
        left, right,
        on=merge_keys,
        how="outer",
        suffixes=("", "_dup")
    ),
    dfs_clean.values()
)


### Remove any rows that contain null values ###
# Replace textual 'nan' with real NaN and assign
replaced = merged.replace('nan', np.nan)
# Keep only rows that have NO nulls in any column
df_all = replaced[~replaced.isnull().any(axis=1)].reset_index(drop=True)


### Deal with the dates ###
# by representing how old each mouse is (days) for various tasks
dob_col = df_all.columns[df_all.columns.str.contains("dob")]
if not dob_col.empty:
  # Extract column
  dob_col = dob_col[0]
  print(f"Found DOB Column: {dob_col} \nCalculating days old for tasks")
  fed_starts = df_all.columns[df_all.columns.str.contains("FED_StartDate")]

  # Convert DOB column once to datetime
  dob_dt = pd.to_datetime(df_all[dob_col], errors='coerce')

  # Loop through each FED_StartDate column and calculate days difference
  for fed_col in fed_starts:
    new_col_name = f"days_old_for_{fed_col}"
    df_all[new_col_name] = (
        pd.to_datetime(df_all[fed_col], errors='coerce') - dob_dt
        ).dt.days
    print(f"→ Created column: {new_col_name}")


### Remove any datetime columns ###
datetime_cols = df_all.select_dtypes(include=['datetime64']).columns
df_all.drop(columns = datetime_cols, inplace=True)


### Check the data ###
for col in df_all.columns:
  # Data types
  col_type = df_all[col].dtypes
  # Null percentages
  col_null = (df_all[col].isna().sum() / (len(df_all[col]) * 100))
  print(f"Name: {col} | Datatype: {col_type} | Percent null: {col_null} ")
#print(df_all.columns.values)
#df_prof(df_all)
#df_all.head()




# ---------- Save CSV ----------
#csv_name = f"PRmetrics_{datetime.now():%Y%m%d_%H%M%S}.csv"
csv_name = f"all_merged_dataframes.csv"

down_df = df_all.to_csv(csv_name, index=False)
#display(HTML(f"<b>✓ Saved PR metrics CSV to:</b> <code>{csv_name}</code>"))

try:
    from google.colab import files as gfiles
except Exception:
    gfiles = None

# Optional download button (works in Colab)
btn = widgets.Button(description=f"Download {os.path.basename(csv_name)}", icon="download")
status = widgets.HTML()
def _dl(_):
    if gfiles is not None:
        status.value = f"Starting download: <code>{os.path.basename(csv_name)}</code>…"
        gfiles.download(csv_name)
    else:
        status.value = f"Saved locally at <code>{csv_name}</code>."
display(btn, status)
btn.on_click(_dl)




In [ ]:
# @title # Prepare data for multiple ML approaches
# @markdown Should prepare data in theory for a RandomForestModel\
# @markdown This will include:
# @markdown 1. One-hot Encoding
# @markdown 2. Split into Training and Testing datasets
# @markdown 3. Normalization of data



from sklearn.preprocessing import LabelEncoder
### Define the classifier for prediction ###
pred_col = ["gene"]

### Define known categorical columns ###
known_objects = ["sex", "genotype"]

### Remove date-time columns ###
datetime_cols = df.select_dtypes(include=['datetime64[ns]']).columns
# Drop the datetime columns
df_no_datetime = df.drop(columns=datetime_cols)

# Drop problem columns
columns_to_drop = [col for col in df_all.columns if
                   not (col.startswith('stim_partner')
                   or col.startswith('stim_weight')
                   or col.startswith('fed3_')
                   or col.startswith('beam_')
                   or col.startswith('fed_')
                   or col.startswith('dob_')
                   or col.startswith('session_type')
                   or col.startswith('time_of_min_')
                   or col.startswith('time_of_max_peak_')
                   or col.startswith('beam_time_of')
                   or col.startswith('unnamed')
                   or col.startswith('weight')
                   )]
df_all = df_all[columns_to_drop]



### Edit the df Meta table
# Define patterns to drop
patterns_to_drop = [
    'stim_partner',
    'stim_weight',
    'fed3_',
    'beam_',
    'fed_',
    'dob_',
    'session_type',
    'time_of_min_',
    'time_of_max_peak_',
    'beam_time_of',
    'unnamed',
    'weight'
]

# Keep rows where the column **does not start with any of these patterns**
df_meta = df_meta[
    ~df_meta['column_name'].astype(str).str.startswith(tuple(patterns_to_drop))
]




### Cast categorical columns ###
df_all[known_objects] = df_all[known_objects].astype('object')




### Begin encoding ###

# encode multiclass prediciton
le = LabelEncoder()
df_all["gene"] = le.fit_transform(df_all["gene"])


# Define ID columns (Columns to Drop)
id_cols = ["mouse_id"]
id_cols.extend(pred_col)


# Define X and y
y = df_all[pred_col]
print(y)

X = df_all.drop(columns=id_cols)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Identify categorical columns
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns

# One-hot encode categorical features
print(f"One-hot encoding the following columns: {list(cat_cols)}")
X_train = pd.get_dummies(X_train, columns=cat_cols, drop_first=True, dtype=bool)
X_test = pd.get_dummies(X_test, columns=cat_cols, drop_first=True, dtype=bool)

# Align columns between train and test
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Scale data
# find numeric
num_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
# find boolean
bool_cols = X_train.select_dtypes(include=['bool']).columns


# fit to only training
scaler.fit(X_train[num_cols])

X_train[num_cols] = scaler.transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])


"""
# Run lasso to determine linked features
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train, y_train)

coefficients = pd.Series(lasso_model.coef_, index=X_train.columns)
selected_features = coefficients[coefficients != 0].index.tolist()
print(selected_features)

# Return only selected features
X_train = X_train[selected_features]
X_test = X_test[selected_features]
"""


# Glimpse training data
X_train.head()
#print(df_prof(X_train))


### create the download for a metafile ###
csv_name2 = f"meta_dataframe.csv"
down_df2 = df_meta.to_csv(csv_name2, index=False)

btn = widgets.Button(description=f"Download {os.path.basename(csv_name2)}", icon="download")
status = widgets.HTML()
def _dl(_):
    if gfiles is not None:
        status.value = f"Starting download: <code>{os.path.basename(csv_name2)}</code>…"
        gfiles.download(csv_name2)
    else:
        status.value = f"Saved locally at <code>{csv_name2}</code>."
display(btn, status)
btn.on_click(_dl)

In [ ]:
#@title # Tune hyperparameters for Random Forest Modeling

# tune hyperparameters
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler()
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso


# Define RF
rf = RandomForestClassifier()

param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# 1) Your Code Below
# initialize the stratified fold
cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)


grid_search = GridSearchCV(
    estimator = rf,
    param_grid = param_grid_rf,
    scoring = "roc_auc_ovr",
    cv = cv,
    n_jobs = 1
)

grid_search.fit(X_train, y_train)

#grid_search
print(grid_search.best_params_)
print(grid_search.best_score_)


In [ ]:
#@title # Fit RF model onto data using best parameters

# refit using best parameters
# Get the best parameters
best_params = grid_search.best_params_

# Re-initialize model with best parameters
final_rf = RandomForestClassifier(**best_params, random_state=42)

# Fit the model on the entire training data
final_rf.fit(X_train, y_train)

In [ ]:
#@title # Perform predictions and check prediction scores

from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Predictions
y_pred = final_rf.predict(X_test)
y_prob = final_rf.predict_proba(X_test)

# Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# ROC-AUC
print("ROC-AUC (OvR):", roc_auc_score(y_test, y_prob, multi_class='ovr'))

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


# Confusion matrix heatmap
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()



In [ ]:
#@title Find and view most important features

import pandas as pd
import numpy as np

feat_imp = pd.DataFrame({
    "feature": X_train.columns,
    "importance": final_rf.feature_importances_
})

feat_imp = feat_imp.sort_values("importance", ascending=False)

print("Most Important features for prediction")
print(feat_imp.head(15))

print(f"\nLeast important features for prediction")
print(feat_imp.tail(15))


In [ ]:
# @title # Correlation Matrix



# Build groups for features linked to


### Drop Keys and object columns ###
pca_df = df_all.drop(columns = merge_keys)
#foo = pd.concat([X_train, X_test])
drops = pca_df.select_dtypes(include=['object', 'bool']).columns
pca_df.drop(columns = drops, inplace=True)


# Plot correlation matrix to understand highly linked features
# Correlation matrix
corr = pca_df.corr()

groups = df_meta["session_type"].unique()
palette = sns.color_palette("tab10", n_colors=len(groups))

# Build a mapping: group -> color
group_colors = {}
for g, color in zip(groups, palette):
    group_colors[g] = color



col_colors = [group_colors[g] for g in df_meta["session_type"]]



sns.clustermap(corr, row_colors=col_colors, col_colors=col_colors,
               row_cluster=False, col_cluster=False, cmap="coolwarm", annot=False)



handles = [mpatches.Patch(color=color, label=grp) for grp, color in group_colors.items()]
plt.legend(handles=handles, bbox_to_anchor=(1,1))

plt.show()


""" This kinda works
# Create a feature -> group mapping
feature_to_group = dict(zip(df_meta["column_name"], df_meta["session_type"]))

# Replace tick labels with group names
xlabels = [feature_to_group[col] for col in corr.columns]
ylabels = [feature_to_group[row] for row in corr.index]

plt.figure(figsize=(6,6))
sns.heatmap(corr, xticklabels=xlabels, yticklabels=ylabels,
            cmap="coolwarm", annot=False, fmt=".2f")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.title("Feature-level correlation with group x-axis labels")
plt.tight_layout()
plt.show()
"""


#Drops high correlation data
# Find upper correlatives
threshold = 0.95
corr_matrix_abs = corr.corr().abs()
upper_tri = corr_matrix_abs.where(pd.DataFrame(np.triu(np.ones(corr_matrix_abs.shape), k=1),
                                                columns=corr_matrix_abs.columns,
                                           index=corr_matrix_abs.index).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]
print(f"Highly correlated features: {to_drop}")
#pca_df = pca_df.drop(columns=to_drop)



In [ ]:
# @title # Attempt UMAP

### Drop Keys and object columns ###
umap_df = df_all.copy()

# drop rows with wt
geno_type = "HET"
umap_df = umap_df[umap_df['genotype'] == geno_type]

# Find sex for labeling later
gender_labels = umap_df['sex']

# get gene labels to plot by gene
labels_encode = umap_df["gene"]
labels = le.inverse_transform(umap_df["gene"])

# Drop the columns that are identifiers
umap_df = umap_df.drop(columns = merge_keys)
print(merge_keys)

# Ensure columns have been dropped
drops = umap_df.select_dtypes(include=['object', 'bool']).columns
umap_df.drop(columns = drops, inplace=True)

# features to scale
features = umap_df.select_dtypes(include=['int64', 'float64']).columns

### Standardize the scale ###
x = umap_df.loc[:, features].values
x = StandardScaler().fit_transform(x)

feat_cols = ['feature'+str(i) for i in range(x.shape[1])]
umap_df = pd.DataFrame(x, columns=feat_cols)

### Create the UMAP embeddings ###
reducer = umap.UMAP(
    n_neighbors=15,
    min_dist=0.1,
    metric="euclidean",
    random_state=42
)

embedding = reducer.fit_transform(umap_df)


# Convert gene labels to colors
unique_genes = np.unique(labels)
palette = sns.color_palette("Pastel2", len(unique_genes))
gene_color_map = dict(zip(unique_genes, palette))

# Male / Female masks
male_mask = (gender_labels == 'M')
female_mask = (gender_labels == 'F')

plt.figure(figsize=(8,6))

### Male UMAP = filled points ###
plt.scatter(
    embedding[male_mask, 0],
    embedding[male_mask, 1],
    s=60,
    c=[gene_color_map[g] for g in labels[male_mask]],
    #edgecolors='black',
    #linewidths=0.5,
    label="Male (filled)"
)

### female UMAP = hollow points ###
plt.scatter(
    embedding[female_mask, 0],
    embedding[female_mask, 1],
    s=60,
    facecolors='none',
    edgecolors=[gene_color_map[g] for g in labels[female_mask]],
    linewidths=3.0,
    label="Female (hollow)"
)

# Gene color legend
for g in unique_genes:
    plt.scatter([], [], color=gene_color_map[g], label=g)
plt.legend(title="Gene", bbox_to_anchor=(1.05, 1), loc='upper left')

### Plot the UMAP combied ###
plt.title(f"UMAP: Colored by Gene for {geno_type}, Filled by Sex")
plt.xlabel("UMAP-1")
plt.ylabel("UMAP-2")
plt.tight_layout()
plt.show()

In [ ]:
# @title K-means & lidean clustering comparison for UMAP embeddings

# Perform k-means clustering and save
kmeans = KMeans(n_clusters=5, random_state=42)
kmean_labels = kmeans.fit_predict(umap_df)

# PCA for visualization
reducer = umap.UMAP(
    n_neighbors=15,
    min_dist=0.1,
    metric="euclidean",
    random_state=42
)
clustering_kmeans_umap = reducer.fit_transform(umap_df)

# Plot our pca data colored by clusters
plt.figure(figsize=(4,4))
plt.scatter(clustering_kmeans_umap[:, 0], clustering_kmeans_umap[:, 1], c=kmean_labels,
            cmap='Pastel2', s=30)
plt.title('K-Means Clusters UMAP')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')

# leiden - Build kNN graph
k = 5
knn_graph = kneighbors_graph(umap_df, k, mode='connectivity', include_self=False)
sources, targets = knn_graph.nonzero()

# Create igraph object
g = ig.Graph(edges=list(zip(sources, targets)), directed=False)
g.simplify()

# Iterate thteough resplutions to find clustering for 5 groups
for res in [0.2, 0.4, 0.6, 0.8, 1.0, 1.2]:
    part = leidenalg.find_partition(
        g,
        leidenalg.RBConfigurationVertexPartition,
        resolution_parameter=res
    )
    print(res, len(set(part.membership)))

res = 1.2
# run Leiden partitioning
partition = leidenalg.find_partition(
    g,
    leidenalg.RBConfigurationVertexPartition,
    resolution_parameter = res
)
leiden_labels = np.array(partition.membership)

# Partition
partition = leidenalg.find_partition(
    g,
    leidenalg.RBConfigurationVertexPartition,
    resolution_parameter=1  # fewer clusters
)

# Plot the Leiden Clustering
plt.figure(figsize=(4,4))
plt.scatter(
    clustering_kmeans_umap[:,0],
    clustering_kmeans_umap[:,1],
    c=leiden_labels,
    s=8,
    alpha=0.8
)
plt.title(f"Leiden Clustering (k = {k}, resolution = {res})")
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.colorbar(label="Cluster")
plt.tight_layout()
plt.show()

In [ ]:
# @title compare raw embeddings of labels to actual labels


# Replace with your real label column
true_labels = labels
#print(f"umap: {true_labels}")
kmean_labels_comp = le.inverse_transform(kmean_labels)
#print(f"kmeans: {le.inverse_transform(kmean_labels)}")
leiden_labels_comp = le.inverse_transform(leiden_labels)
#print(f"lendien: {le.inverse_transform(leiden_labels)}")



print("=== Adjusted Rand Index (ARI) ===")
print("KMeans vs Actual:", adjusted_rand_score(true_labels, kmean_labels_comp))
print("Leiden vs Actual:", adjusted_rand_score(true_labels, leiden_labels_comp))

print("\n=== Normalized Mutual Information (NMI) ===")
print("KMeans vs Actual:", normalized_mutual_info_score(true_labels, kmean_labels_comp))
print("Leiden vs Actual:", normalized_mutual_info_score(true_labels, leiden_labels_comp))


cm_kmeans = pd.DataFrame(
    confusion_matrix(true_labels, kmean_labels_comp),
    index=[f"True_{x}" for x in sorted(set(true_labels))],
    columns=[f"KMeans_{x}" for x in sorted(set(kmean_labels_comp))]
)

cm_leiden = pd.DataFrame(
    confusion_matrix(true_labels, leiden_labels_comp),
    index=[f"True_{x}" for x in sorted(set(true_labels))],
    columns=[f"Leiden_{x}" for x in sorted(set(leiden_labels_comp))]
)

print("\n=== Confusion Matrix: KMeans ===")
#print(cm_kmeans)
plt.figure(figsize=(8,6))
sns.heatmap(cm_kmeans, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix: True Labels vs K-Means Clusters")
plt.ylabel("True Labels")
plt.xlabel("Kmeans Clusters")
plt.tight_layout()
plt.show()

print("\n=== Confusion Matrix: Leiden ===")
#print(cm_leiden)
plt.figure(figsize=(8,6))
sns.heatmap(cm_leiden, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix: True Labels vs K-Means Clusters")
plt.ylabel("True Labels")
plt.xlabel("Kmeans Clusters")
plt.tight_layout()
plt.show()



In [ ]:
# @title Align using Algorithm and compare True vs False


#### Assign labels ####
# True labels
true_labels = np.array(labels_encode)
print(f"# of labels in true: {np.unique(true_labels)}")

# Predicted clusters
kmeans_labels = kmeans.labels_
print(f"# of k-means labels: {np.unique(kmeans_labels)}")
leiden_labels = leiden_labels # from Leiden
print(f"# of leiden labels: {np.unique(leiden_labels)}")

#### Optimal label mapping using Hungarian algorithm ####
def map_clusters_to_labels(true, pred):
    cm = confusion_matrix(true, pred)
    row_ind, col_ind = linear_sum_assignment(-cm)  # maximize matches
    mapping = dict(zip(col_ind, row_ind))
    mapped_pred = np.array([mapping[x] for x in pred])
    return mapped_pred

kmeans_labels_mapped = map_clusters_to_labels(true_labels, kmeans_labels)
leiden_labels_mapped = map_clusters_to_labels(true_labels, leiden_labels)

# ==============================
# 3. Metrics
# ==============================
print("=== Adjusted Rand Index (ARI) ===")
print("KMeans vs Actual:", adjusted_rand_score(true_labels, kmeans_labels))
print("Leiden vs Actual:", adjusted_rand_score(true_labels, leiden_labels))

print("\n=== Normalized Mutual Information (NMI) ===")
print("KMeans vs Actual:", normalized_mutual_info_score(true_labels, kmeans_labels))
print("Leiden vs Actual:", normalized_mutual_info_score(true_labels, leiden_labels))

# ==============================
# 4. Confusion matrices
# ==============================
def plot_confusion(true, pred, title):
    cm = pd.DataFrame(
        confusion_matrix(true, pred),
        index=[f"True_{x}" for x in sorted(set(true))],
        columns=[f"Pred_{x}" for x in sorted(set(pred))]
    )
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(title)
    plt.ylabel("True Labels")
    plt.xlabel("Predicted Clusters")
    plt.tight_layout()
    plt.show()
    return cm

true_labels_plotting = le.inverse_transform(true_labels)
kmeans_labels_mapped_plotting = le.inverse_transform(kmeans_labels_mapped)
leiden_labels_mapped_plotting = le.inverse_transform(leiden_labels_mapped)

cm_kmeans = plot_confusion(true_labels_plotting, kmeans_labels_mapped_plotting, "Confusion Matrix: True vs KMeans")
cm_leiden = plot_confusion(true_labels_plotting, leiden_labels_mapped_plotting, "Confusion Matrix: True vs Leiden")

#### Cluster purity (optional) ####
def cluster_purity(true, pred):
    import pandas as pd
    import numpy as np
    results = []
    for c in np.unique(pred):
        idx = np.where(pred == c)[0]
        true_subset = true[idx]
        majority_label = pd.Series(true_subset).mode()[0]
        purity = (true_subset == majority_label).mean()
        results.append([c, len(idx), majority_label, purity])
    return pd.DataFrame(results, columns=["Cluster", "Size", "Majority Label", "Purity"])

print("\n=== KMeans Cluster Purity ===")
print(cluster_purity(true_labels_plotting, kmeans_labels_mapped_plotting))

print("\n=== Leiden Cluster Purity ===")
print(cluster_purity(true_labels_plotting, leiden_labels_mapped_plotting))

In [ ]:
# @title # UMAP plotting for gene and genotype together

### Drop Keys and object columns ###
umap_df = df_all.copy()

geno_type = ["WT", "HET"]
umap_df = umap_df[umap_df['genotype'].isin(geno_type)]

# get gene labels to plot by gene
umap_df["gene"] = le.inverse_transform(umap_df["gene"])

# Create a column that mixes genotype and gene
umap_df['gene_genotype'] = umap_df['gene'] + "_" + umap_df['genotype']


# Find sex for labeling later
gender_labels = umap_df['sex']

# get gene labels to plot by gene
#labels = le.inverse_transform(umap_df["gene"])
labels = umap_df["gene_genotype"]
umap_df.drop('gene_genotype', axis = 1, inplace=True)


# Drop the columns that are needed
umap_df = umap_df.drop(columns = merge_keys)
print(merge_keys)

# Ensure columns have been dropped
drops = umap_df.select_dtypes(include=['object', 'bool']).columns
umap_df.drop(columns = drops, inplace=True)


# features to scale
features = umap_df.select_dtypes(include=['int64', 'float64']).columns

### Standardize the scale ###
x = umap_df.loc[:, features].values
x = StandardScaler().fit_transform(x)

feat_cols = ['feature'+str(i) for i in range(x.shape[1])]
umap_df = pd.DataFrame(x, columns=feat_cols)


umap_df.head()


### Create the UMAP embeddings ###
reducer = umap.UMAP(
    n_neighbors=15,
    min_dist=0.1,
    metric="euclidean",
    random_state=42
)
embedding = reducer.fit_transform(umap_df)


# Convert gene labels to colors
unique_genes = np.unique(labels)
palette = sns.color_palette("Paired", len(unique_genes))
gene_color_map = dict(zip(unique_genes, palette))

# Male / Female masks
male_mask = (gender_labels == 'M')
female_mask = (gender_labels == 'F')

plt.figure(figsize=(8,6))

### Male UMAP = filled points ###
plt.scatter(
    embedding[male_mask, 0],
    embedding[male_mask, 1],
    s=60,
    c=[gene_color_map[g] for g in labels[male_mask]],
    #edgecolors='black',
    #linewidths=0.5,
    label="Male (filled)"
)

### female UMAP = hollow points ###
plt.scatter(
    embedding[female_mask, 0],
    embedding[female_mask, 1],
    s=60,
    facecolors='none',
    edgecolors=[gene_color_map[g] for g in labels[female_mask]],
    linewidths=3.0,
    label="Female (hollow)"
)

# Gene color legend
for g in unique_genes:
    plt.scatter([], [], color=gene_color_map[g], label=g)
plt.legend(title="Gene", bbox_to_anchor=(1.05, 1), loc='upper left')

### Plot the UMAP combied ###
plt.title(f"UMAP: Colored by Gene and Genotype, Filled by Sex")
plt.xlabel("UMAP-1")
plt.ylabel("UMAP-2")
plt.tight_layout()
plt.show()


In [ ]:
# @title Conduct mutliple UMAPS based on different tests

# create a copy of the meta table
source_df = df_meta.copy()

# define the columns we are zgoing to keep
keep_cols = ['gene', 'sex', 'genotype']

# Iterate through all the test type to generate umaps
for source in source_df['session_type'].unique():
    print(f"Sourcing UMAP from: {source}" )

    # Make sure umap_df has only columns from the particular session type
    cols_for_source = source_df.loc[
        source_df['session_type'] == source,
        'column_name'
    ].tolist()

   # Keep only matching columns (and optional metadata columns)
    selected_cols = [c for c in cols_for_source if c in df_all.columns]
    umap_df = df_all[keep_cols + selected_cols].copy()

    print("Columns kept:", keep_cols, selected_cols)

    # keep only HET mice
    geno_type = "HET"
    umap_df = umap_df[umap_df['genotype'] == geno_type]

    # Find sex for labeling later
    gender_labels = umap_df['sex']

    # get gene labels to plot by gene
    labels = le.inverse_transform(umap_df["gene"])

    # Drop the columns that are labels
    umap_df = umap_df.drop(columns = keep_cols)

    # Ensure columns have been dropped
    drops = umap_df.select_dtypes(include=['object', 'bool']).columns
    umap_df.drop(columns = drops, inplace=True)

    # features to scale
    features = umap_df.select_dtypes(include=['int64', 'float64']).columns

    ### Standardize the scale ###
    x = umap_df.loc[:, features].values
    x = StandardScaler().fit_transform(x)

    feat_cols = ['feature'+str(i) for i in range(x.shape[1])]
    umap_df = pd.DataFrame(x, columns=feat_cols)

    ### Create the UMAP embeddings ###
    reducer = umap.UMAP(
        n_neighbors=15,
        min_dist=0.1,
        metric="euclidean",
        random_state=42
    )
    embedding = reducer.fit_transform(umap_df)


    # Convert gene labels to colors
    unique_genes = np.unique(labels)
    palette = sns.color_palette("Paired", len(unique_genes))
    gene_color_map = dict(zip(unique_genes, palette))

    # Male / Female masks
    male_mask = (gender_labels == 'M')
    female_mask = (gender_labels == 'F')

    plt.figure(figsize=(8,6))

    ### Male UMAP = filled points ###
    plt.scatter(
        embedding[male_mask, 0],
        embedding[male_mask, 1],
        s=60,
        c=[gene_color_map[g] for g in labels[male_mask]],
        #edgecolors='black',
        #linewidths=0.5,
        label="Male (filled)"
    )

    ### female UMAP = hollow points ###
    plt.scatter(
        embedding[female_mask, 0],
        embedding[female_mask, 1],
        s=60,
        facecolors='none',
        edgecolors=[gene_color_map[g] for g in labels[female_mask]],
        linewidths=3.0,
        label="Female (hollow)"
    )

    # Gene color legend
    for g in unique_genes:
        plt.scatter([], [], color=gene_color_map[g], label=g)
    plt.legend(title="Gene", bbox_to_anchor=(1.05, 1), loc='upper left')

    ### Plot the UMAP combied ###
    plt.title(f"UMAP: Colored by Gene, Filled by Sex, for {geno_type}, Session Type: {source}")
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.tight_layout()
    plt.show()


In [ ]:
# @title Conduct mutliple UMAPS based on different tests plotting both Gene and Genotype

# create a copy of the meta table
source_df = df_meta.copy()




# define the columns we are going to keep for now
keep_cols = ['gene', 'sex', 'genotype']

# Iterate through all the test type to generate umaps
for source in source_df['session_type'].unique():
    print(f"Sourcing UMAP from: {source}" )

    # Make sure umap_df has only columns from the particular session type
    cols_for_source = source_df.loc[
        source_df['session_type'] == source,
        'column_name'
    ].tolist()



   # Keep only matching columns (and optional metadata columns)
    selected_cols = [c for c in cols_for_source if c in df_all.columns]
    umap_df = df_all[keep_cols + selected_cols].copy()

    print("Columns kept:", keep_cols, selected_cols)

    # get gene labels to plot by gene
    umap_df["gene"] = le.inverse_transform(umap_df["gene"])

    # Create a column that mixes genotype and gene
    umap_df['gene_genotype'] = umap_df['gene'] + "_" + umap_df['genotype']


    # Find sex for labeling later
    gender_labels = umap_df['sex']

    # get gene labels to plot by gene
    #labels = le.inverse_transform(umap_df["gene"])
    labels = umap_df["gene_genotype"]
    umap_df.drop(columns = ["gene_genotype"], inplace=True)

    # Drop the columns that are labels
    umap_df = umap_df.drop(columns = keep_cols)

    # Ensure columns have been dropped
    drops = umap_df.select_dtypes(include=['object', 'bool']).columns
    umap_df.drop(columns = drops, inplace=True)

    # features to scale
    features = umap_df.select_dtypes(include=['int64', 'float64']).columns

    ### Standardize the scale ###
    x = umap_df.loc[:, features].values
    x = StandardScaler().fit_transform(x)

    feat_cols = ['feature'+str(i) for i in range(x.shape[1])]
    umap_df = pd.DataFrame(x, columns=feat_cols)

    ### Create the UMAP embeddings ###
    reducer = umap.UMAP(
        n_neighbors=15,
        min_dist=0.1,
        metric="euclidean",
        random_state=42
    )
    embedding = reducer.fit_transform(umap_df)


    # Convert gene labels to colors
    unique_genes = np.unique(labels)
    palette = sns.color_palette("Paired", len(unique_genes))
    gene_color_map = dict(zip(unique_genes, palette))

    # Male / Female masks
    male_mask = (gender_labels == 'M')
    female_mask = (gender_labels == 'F')

    plt.figure(figsize=(8,6))

    ### Male UMAP = filled points ###
    plt.scatter(
        embedding[male_mask, 0],
        embedding[male_mask, 1],
        s=60,
        c=[gene_color_map[g] for g in labels[male_mask]],
        #edgecolors='black',
        #linewidths=0.5,
        label="Male (filled)"
    )

    ### female UMAP = hollow points ###
    plt.scatter(
        embedding[female_mask, 0],
        embedding[female_mask, 1],
        s=60,
        facecolors='none',
        edgecolors=[gene_color_map[g] for g in labels[female_mask]],
        linewidths=3.0,
        label="Female (hollow)"
    )

    # Gene color legend
    for g in unique_genes:
        plt.scatter([], [], color=gene_color_map[g], label=g)
    plt.legend(title="Gene", bbox_to_anchor=(1.05, 1), loc='upper left')

    ### Plot the UMAP combied ###
    plt.title(f"UMAP: Colored by Gene, Filled by Sex | Session Type: {source}")
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.tight_layout()
    plt.show()

In [ ]:
# @title # Conduct PCA Plotting Colored by Various Categoricals
# make a copy from the df_all
pca_df = df_all.copy()

print(pca_df['genotype'].unique())

# keep only HETs
geno_type = "HET"
pca_df = pca_df[pca_df['genotype'] == geno_type]

# get gene labels to plot by gene
pca_labels_encode = pca_df["gene"]
pca_labels = le.inverse_transform(pca_df["gene"])


# Drop categoricals
drops = pca_df.select_dtypes(include=['object', 'bool']).columns
pca_df.drop(columns = drops, inplace=True)

# find most important features
##non_feat = feat_imp[feat_imp['importance'] < 0.02]
#non_feat_drops = list(non_feat['feature'])
#print(non_feat_drops)
#cols_to_drop = [c for c in non_feat_drops if c in pca_df.columns]
#pca_df.drop(columns=cols_to_drop, inplace=True)

features = pca_df.select_dtypes(include=['int64', 'float64']).columns
print(len(features))

### Standardize the scale ###
x = pca_df.loc[:, features].values
x = StandardScaler().fit_transform(x)
feat_cols = ['feature'+str(i) for i in range(x.shape[1])]
pca_df = pd.DataFrame(x, columns=feat_cols)

### PCA Plotting ###
pca = PCA(n_components=2)
clustering_df_pca = pca.fit_transform(pca_df)
clustering_df_pca = pd.DataFrame(data = clustering_df_pca,
                                 columns = ['principal component 1', 'principal component 2'])
print('Explained variability per principal component: {}'.format(pca.explained_variance_ratio_))

unique_genes_pca = np.unique(pca_labels)
palette = sns.color_palette("Paired", len(unique_genes_pca))
gene_color_map_pca = dict(zip(unique_genes_pca, palette))

plt.figure(figsize=(4,4))
plt.scatter(clustering_df_pca[:, 0], clustering_df_pca[:, 1],
            c=unique_genes_pca, cmap='viridis', s=30)
plt.title('K-Means Clusters (PCA 2D Projection)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')


"""
# Colummns for coloring
categorical_columns = ['gene']

for cat_col in categorical_columns:
    plt.figure(figsize=(6,6))
    plt.title(f"PCA colored by {cat_col}", fontsize=16)
    plt.xlabel("Principal Component 1", fontsize=14)
    plt.ylabel("Principal Component 2", fontsize=14)

    ### Get unique groups for column ###
    targets = df_all[cat_col].dropna().unique()

    # Generate colors
    cmap = plt.cm.get_cmap('Paired', len(targets))
    colors = [cmap(i) for i in range(len(targets))]

    # Scatter for each group
    for i, target in enumerate(targets):
        indicesToKeep = df_all[cat_col] == target
        plt.scatter(
            clustering_df_pca.loc[indicesToKeep, 'principal component 1'],
            clustering_df_pca.loc[indicesToKeep, 'principal component 2'],
            color=colors[i],
            s=50,
            alpha=0.7,
            edgecolors='k',
            label=str(target)
        )
    plt.legend(title=cat_col, fontsize=9, title_fontsize=11)
    plt.tight_layout()
    plt.show()"""

In [ ]:
# @title Perform unsupervised PCA  k-means & KNN clustering
kmeans = KMeans(n_clusters=5, random_state=42)
labels = kmeans.fit_predict(pca_df)

# PCA for visualization
pca = PCA(n_components=2)
clustering_df_pca = pca.fit_transform(pca_df)

# Plot our pca data colored by clusters
plt.figure(figsize=(4,4))
plt.scatter(clustering_df_pca[:, 0], clustering_df_pca[:, 1], c=labels,
            cmap='viridis', s=30)
plt.title('K-Means Clusters (PCA 2D Projection)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# leiden - Build kNN graph
k = 5
knn_graph = kneighbors_graph(pca_df, k, mode='connectivity', include_self=False)
sources, targets = knn_graph.nonzero()

# Create igraph object
g = ig.Graph(edges=list(zip(sources, targets)), directed=False)
g.simplify()

# Iterate thteough resplutions to find clustering for 5 groups
for res in [0.2, 0.4, 0.6, 0.8, 1.0, 1.2]:
    part = leidenalg.find_partition(
        g,
        leidenalg.RBConfigurationVertexPartition,
        resolution_parameter=res
    )
    print(res, len(set(part.membership)))

res = 1.2
# run Leiden partitioning
partition = leidenalg.find_partition(
    g,
    leidenalg.RBConfigurationVertexPartition,
    resolution_parameter = res
)
leiden_labels = np.array(partition.membership)

# Partition
partition = leidenalg.find_partition(
    g,
    leidenalg.RBConfigurationVertexPartition,
    resolution_parameter=1  # fewer clusters
)
